In [ ]:
import csv
import tempfile
import wandb
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from vqgan_jax.modeling_flax_vqgan import VQModel
from transformers import BartTokenizer, CLIPProcessor, FlaxCLIPModel
from dalle_mini.text import TextNormalizer

In [ ]:
wandb_runs = ['rjf3rycy']
VQGAN_REPO, VQGAN_COMMIT_ID = 'dalle-mini/vqgan_imagenet_f16_16384', None
normalize_text = True

In [ ]:
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)
clip = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
with open('samples.csv', newline='', encoding='utf8') as f:
    reader = csv.reader(f)
    for row in reader:
        breakpoint()

In [ ]:
wandb_run = wandb_runs[0]
api = wandb.Api()

In [ ]:
try:
    versions = api.artifact_versions(type_name='bart_model', name=f'dalle-mini/dalle-mini/model-{wandb_run}', per_page=10000)
except:
    versions = []

In [ ]:
versions, len(versions)

In [ ]:
versions = sorted(versions, key=lambda x: int(x.version[1:]))

In [ ]:
versions

In [ ]:
artifact = versions[0]

In [ ]:
version = int(artifact.version[1:])

In [ ]:
version

In [ ]:
# retrieve training run
training_run = api.run(f'dalle-mini/dalle-mini/{wandb_run}')
config = training_run.config

In [ ]:
# see summary metrics
training_run.summary

In [ ]:
# retrieve inference run details
def get_last_version_inference(run_id):
    try:
        inference_run = api.run(f'dalle-mini/dalle-mini/inference-{run_id}')
        return inference_run.summary.get('_step', None)
    except:
        return None

In [ ]:
last_version_inference = get_last_version_inference(wandb_run)

In [ ]:
if last_version_inference is None:
    assert version == 0
else:
    assert version == last_version_inference + 1

In [ ]:
run = wandb.init(job_type='inference', config=config, id=f'inference-{wandb_run}', resume='allow')

In [ ]:
tmp_f.cleanup
tmp_f = tempfile.TemporaryDirectory()
tmp = tmp_f.name
#TODO: use context manager

In [ ]:
# remove tmp
tmp_f.cleanup()

In [ ]:
artifact = run.use_artifact(artifact)

In [ ]:
# only download required files
for f in ['config.json', 'flax_model.msgpack', 'merges.txt', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.json']:
    artifact.get_path(f).download(tmp)

In [ ]:
# we verify all the files are present
from pathlib import Path
list(Path(tmp).glob('*'))

In [ ]:
tokenizer = BartTokenizer.from_pretrained(tmp)
model = CustomFlaxBartForConditionalGeneration.from_pretrained(tmp)

In [ ]:
wandb.finish()